## BASE DE DONNEES VECTORIELLE ET EMBEDING

## ETAPE 1 CHUNKING

In [2]:
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def load_and_split_pdf_simple(pdf_path: str):
    """
    Charge tout le PDF en un seul Document, puis le découpe en chunks
    avec chunk_size=1000 et chunk_overlap=100 (en caractères).
    """
    # 1. Extraire tout le texte du PDF
    reader = PdfReader(pdf_path)
    full_text = ""
    for i, page in enumerate(reader.pages):
        text = page.extract_text() or ""
        full_text += text + f"\n\n[PAGE {i+1}]\n\n"  # Ajout de marqueur de page

    # 2. Créer un seul Document
    doc = Document(page_content=full_text.strip().replace('. .',''), metadata={"source": pdf_path})

    # 3. Découper avec RecursiveCharacterTextSplitter (par défaut : len = caractères)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=['.'],
        keep_separator=True  # Garde les séparateurs pour éviter de couper au milieu d'une phrase
    )

    # 4. Splitter
    chunks = text_splitter.split_documents([doc])

    return chunks

# Utilisation
pdf_chunks = load_and_split_pdf_simple("embedding_course_by_koulou.pdf")




In [3]:
pdf_chunks

[Document(page_content='L’INTELLIGENCE ARTIFICIELLE\nDes fondements théoriques aux bases de données vectorielles\nAuteur : KOULOU crepin\nDépartement de Data Science\n8 janvier 2026\n\n[PAGE 1]\n\nRésumé\nCe rapport présente une analyse approfondie de l’état actuel de l’Intelligence Artiﬁcielle.\nNous explorerons d’abord les fondements mathématiques de l’apprentissage automatique\n(Machine Learning) et des réseaux de neurones profonds (Deep Learning). Une attention\nparticulière sera portée aux développements récents concernant les Grands Modèles de\nLangage (LLM) et l’infrastructure critique qui les soutient, notamment les bases de don-\nnées vectorielles (FAISS, ChromaDB, Qdrant) et les mécanismes de similarité. Enﬁn, nous\naborderons les déﬁs éthiques et les perspectives d’avenir.\n\n[PAGE 2]\n\nTable des matières\n1 Introduction 2\n1.1 Contexte Historique                . 2\n1.2 Déﬁnitions et Typologie               2\n2 Fondements Mathématiques du Machine Learning 4\n2.1 L’Apprent


<img src="faiss chroma.jpg" alt="Logo Python" width="1800" height="600">


<div style="display: flex; gap: 20px;">

  <!-- Colonne 1 -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 1. Introduction Générale : Les Bases de Données Vectorielles

  **Qu'est-ce que c'est ?**  
  Une base de données vectorielle est un système optimisé pour stocker et interroger des *vecteurs* (embeddings).

  **Pourquoi en avons-nous besoin ?**  
  Les bases traditionnelles (SQL, NoSQL) gèrent bien les correspondances exactes, mais échouent sur la *similarité*.  

  Avec l’essor de l’IA, les données complexes (texte, images, audio) sont transformées en vecteurs.  
  ✨ Des concepts proches se retrouvent mathématiquement voisins dans un espace multidimensionnel.  

  👉 Fonction principale : **Vector Search**.  
  Exemple : un vecteur "chat" renvoie les images les plus proches d’un chat, même sans mot-clé.  

  C’est le cœur des applications modernes comme le **RAG (Retrieval-Augmented Generation)**, qui permet aux LLM d’accéder à vos documents.  

  </div>

  <!-- Colonne 2 -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 2. FAISS et ChromaDB : Solutions orientées "Développement Local"

  **FAISS (Facebook AI Similarity Search)**  
  - Origine : Meta (Facebook AI Research).  
  - Nature : Bibliothèque C++/Python, ultra performante.  
  - Forces :  
    - 🚀 Recherche sur milliards de vecteurs.  
    - ⚡ Optimisation GPU.  
  - Faiblesses :  
    - Bas niveau, pas de stockage persistant.  
    - Risque de perte si non sauvegardé.  

  **ChromaDB**  
  - Origine : Projet Open Source récent.  
  - Nature : Base vectorielle simple et rapide.  
  - Forces :  
    - 🎯 Installation facile (`pip install chromadb`).  
    - 🔄 Vectorisation intégrée.  
  - Faiblesses :  
    - Idéal pour petits/moyens projets.  
    - Moins robuste pour téraoctets en production.  

  </div>

</div>


## FAISS

In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

def build_faiss_index(chunks):
    """
    Construit un index FAISS à partir de chunks déjà préparés.
    Utilise un modèle gratuit Hugging Face pour générer les embeddings.
    """
    # 1. Créer les embeddings avec Hugging Face
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"  # modèle open-source ou openaiembeddings
    )

    # 2. Construire l’index FAISS
    vectorstore = FAISS.from_documents(chunks, embeddings)

    return vectorstore



In [6]:
# Supposons que tu as déjà tes chunks :
# chunks = text_splitter.split_documents(documents)

faiss_index = build_faiss_index(pdf_chunks)

# Faire une requête
query = "Explique-moi la différence entre FAISS et ChromaDB"
results = faiss_index.similarity_search(query, k=10)

# Afficher les résultats
for i, res in enumerate(results):
    print(f"--- Résultat {i+1} ---")
    print(res.page_content[:500], "...")


--- Résultat 1 ---
.
8

[PAGE 10]

État de l’art Rapport IA 2024
4.3.1 Comparatif Technique
— FAISS (Meta): Bibliothèque bas niveau, optimisée pour la recherche par simila-
rité dense et le clustering. Utilise l’indexation IVF.
— ChromaDB : Solution open-source conviviale pour les développeurs, idéale pour
les applications LLM légères et le prototypage.
— Pinecone / Qdrant: Solutions gérées (SaaS) ou robustes (Rust) pour la produc-
tion à grande échelle.
4.3.2 Exemple d’implémentation (Python)
Voici comment on ini ...
--- Résultat 2 ---
. Dans ce contexte, les réseaux de neurones profonds et les
bases de données vectorielles jouent un rôle crucial pour traiter des informations non
structurées. Ce paragraphe sert de remplissage pour simuler le volume de texte du rap-
port ﬁnal, permettant d’apprécier la mise en page sans être distrait par du latin. Les
avancées technologiques récentes, notamment les architectures Transformers, ont ouvert
la voie aux modèles de langage massifs (LLM).
4.2

<img src="reranking.jpg" alt="Logo Python" width="1800" height="600">


## Re-Ranking

<div style="display: flex; gap: 20px;">

  <!-- Colonne 1 -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 1. Reranking : Concept et Pourquoi l’utiliser

  **Qu'est‑ce que le reranking ?**  
  Le *reranking* est une étape secondaire qui réordonne une liste de résultats initialement récupérés (par ex. via FAISS) en utilisant un modèle plus fin ou une méthode différente pour mieux estimer la pertinence.

  **Pourquoi l’ajouter ?**  
  - Les recherches vectorielles (FAISS, ChromaDB) fournissent un bon *précroisement* basé sur la proximité dans l’espace des embeddings.  
  - Le reranking applique une évaluation sémantique plus riche (modèles de rerank, cross-encoders) pour corriger les faux positifs et améliorer la précision en tête de liste.

  **Flux typique**  
  1. **Retrieval** : récupérer top‑k via FAISS (rapide).  
  2. **Rerank** : envoyer ces k documents + la requête à un modèle de reranking.  
  3. **Réordonnancement** : trier par score de rerank et retourner top‑n final.

  **Avantages**  
  - 🔍 Meilleure précision en tête de résultats.  
  - ⚖️ Moins de bruit : on réduit les documents proches mais non pertinents.  
  - 💡 Complémentarité : combine vitesse (FAISS) et finesse (reranker).

  **Limites**  
  - ⏱️ Coût et latence supplémentaires (appel au modèle de rerank).  
  - 💸 Coût monétaire si le reranker est un service payant.  
  - 🔁 Nécessite un bon prétraitement (nettoyage, découpage) pour être efficace.

  **Quand l’utiliser ?**  
  - Quand la qualité des 3–5 premiers résultats est critique (chatbot, RAG, FAQ).  
  - Quand la base contient beaucoup de documents proches sémantiquement mais peu pertinents.

  </div>

  <!-- Colonne 2 -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 2. Cohere et Reranking : Mise en œuvre pratique

  **Pourquoi Cohere pour le rerank ?**  
  Cohere propose des modèles dédiés au *reranking* (cross‑encoder style) qui comparent directement la requête et chaque document pour produire un score de pertinence fin.

  **Pattern d’intégration**  
  - **Étape 1** : FAISS → `similarity_search(query, k=K)`  
  - **Étape 2** : Extraire `page_content` des K résultats.  
  - **Étape 3** : Appel Cohere Rerank avec `query` + `documents`.  
  - **Étape 4** : Réordonner les objets FAISS selon `relevance_score` renvoyé par Cohere.

  **Bonnes pratiques**  
  - **Choisir le modèle** : `rerank-multilingual` si documents en français; `rerank-english` pour l’anglais.  
  - **Limiter K** : 10–50 selon latence/coût.  
  - **Fallback** : prévoir un retour aux résultats FAISS si l’API échoue.  
  - **Conserver métadonnées** : renvoyer l’objet FAISS original pour garder source, offset, page, etc.  
  - **Batching** : grouper les appels si possible pour réduire latence et coût.

  **Exemples d’améliorations observables**  
  - Augmentation de la précision @1 et @3.  
  - Réduction des hallucinations dans les réponses RAG.  
  - Meilleure cohérence des extraits utilisés pour la génération.

  **Risques et mitigations**  
  - **Latence** → mettre en cache les reranks fréquents.  
  - **Coût** → n’utiliser le rerank que pour les requêtes critiques.  
  - **Biais** → tester le reranker sur jeux de requêtes représentatifs.

  **Astuce rapide**  
  Combine un score FAISS (similitude) et le score Cohere (rerank) via une pondération pour tirer parti des deux signaux avant le tri final.

  </div>

</div>

In [ ]:
# === Configuration globale ===
# Remplace par ta clé API Cohere
COHERE_API_KEY = "votre cle api cohere"  # ⚠️ À remplacer ou à charger depuis .env / config


In [12]:
import cohere
from typing import List, Dict, Any, Optional


# Initialiser le client Cohere
co = cohere.Client(COHERE_API_KEY)

# === Fonction principale : Rerank avec Cohere ===
def rerank_with_cohere(
    query: str,
    faiss_results: List[Any],
    top_n: int = 5,
    model: str = "rerank-multilingual-v2.0",
    timeout: int = 30
) -> List[Dict[str, Any]]:
    """
    Rerank les résultats FAISS avec l'API Cohere Rerank.
    
    Args:
        query (str): Requête utilisateur.
        faiss_results (List[Any]): Liste d'objets retournés par `faiss_index.similarity_search`.
        top_n (int): Nombre de résultats à retourner après reranking.
        model (str): Modèle Cohere à utiliser (ex: "rerank-english-v3.0").
        timeout (int): Timeout en secondes pour l'appel API.

    Returns:
        List[Dict]: Liste d'éléments contenant :
            - 'doc': objet FAISS original
            - 'original_index': index dans la liste d'origine
            - 'score': score de pertinence (float) ou None si erreur
    """
    if not faiss_results:
        return []

    # Extraire les textes des documents + garder trace des indices
    documents = []
    for i, res in enumerate(faiss_results):
        text = getattr(res, "page_content", None) or getattr(res, "content", None)
        if not text:
            text = str(res)  # fallback
        documents.append(text)

    # Appel à l'API Cohere avec gestion d'erreur
    try:
        response = co.rerank(
            model=model,
            query=query,
            documents=documents,
            top_n=min(top_n, len(documents)),
            timeout=timeout
        )
    except Exception as e:
        print(f"[⚠️ Warning] Cohere rerank failed: {e}")
        # Retourne les premiers résultats sans reranking
        return [
            {"doc": faiss_results[i], "original_index": i, "score": None}
            for i in range(min(top_n, len(faiss_results)))
        ]

    # Extraire les résultats (compatible avec différentes versions du SDK)
    results_list = getattr(response, "results", None)
    if results_list is None:
        results_list = response  # fallback pour anciennes versions

    # Construire la liste rerankée
    reranked = []
    for item in results_list:
        idx = getattr(item, "index", None)
        score = getattr(item, "relevance_score", None) or getattr(item, "score", None)

        if idx is None or idx >= len(faiss_results):
            continue  # Ignorer les éléments invalides

        reranked.append({
            "doc": faiss_results[idx],
            "original_index": idx,
            "score": float(score) if score is not None else None
        })

    return reranked


In [15]:

# === Exemple d'utilisation ===
# ⚠️ Assure-toi que `faiss_index` est défini avant cet appel
# Exemple fictif :
# from some_module import faiss_index

query = "Explique-moi la différence entre FAISS et ChromaDB"
faiss_results = faiss_index.similarity_search(query, k=10)

reranked_results = rerank_with_cohere(query, faiss_results, top_n=5)

print(len(reranked_results), "résultats après reranking avec Cohere :")

[⚠️ Warning] Cohere rerank failed: rerank() got an unexpected keyword argument 'timeout'
5 résultats après reranking avec Cohere :


In [16]:
reranked_results

[{'doc': Document(page_content='.\n8\n\n[PAGE 10]\n\nÉtat de l’art Rapport IA 2024\n4.3.1 Comparatif Technique\n— FAISS (Meta): Bibliothèque bas niveau, optimisée pour la recherche par simila-\nrité dense et le clustering. Utilise l’indexation IVF.\n— ChromaDB : Solution open-source conviviale pour les développeurs, idéale pour\nles applications LLM légères et le prototypage.\n— Pinecone / Qdrant: Solutions gérées (SaaS) ou robustes (Rust) pour la produc-\ntion à grande échelle.\n4.3.2 Exemple d’implémentation (Python)\nVoici comment on initialise une recherche simple avec la librairie ChromaDB :\n1 import chromadb\n2\n3 # Initialisation du client (en mé moire pour l’ exemple )\n4 client = chromadb . Client ()\n5 collection = client . create_collection (" mon_dataset_ia ")\n6\n7 # Ajout de documents ( texte brut + mé tadonn ées)\n8 collection', metadata={'source': 'embedding_course_by_koulou.pdf'}),
  'original_index': 0,
  'score': None},
 {'doc': Document(page_content='. Dans ce cont

In [14]:


print("\n=== Résultats Rerankés ===\n")
for i, res in enumerate(reranked_results, 1):
    score_str = f"{res['score']:.4f}" if res['score'] is not None else "N/A"
    print(f"--- Résultat {i} --- (Score: {score_str})")
    content = getattr(res["doc"], "page_content", "") or getattr(res["doc"], "content", "")
    print(content[:500], "...")


=== Résultats Rerankés ===

--- Résultat 1 --- (Score: N/A)
.
8

[PAGE 10]

État de l’art Rapport IA 2024
4.3.1 Comparatif Technique
— FAISS (Meta): Bibliothèque bas niveau, optimisée pour la recherche par simila-
rité dense et le clustering. Utilise l’indexation IVF.
— ChromaDB : Solution open-source conviviale pour les développeurs, idéale pour
les applications LLM légères et le prototypage.
— Pinecone / Qdrant: Solutions gérées (SaaS) ou robustes (Rust) pour la produc-
tion à grande échelle.
4.3.2 Exemple d’implémentation (Python)
Voici comment on ini ...
--- Résultat 2 --- (Score: N/A)
. Dans ce contexte, les réseaux de neurones profonds et les
bases de données vectorielles jouent un rôle crucial pour traiter des informations non
structurées. Ce paragraphe sert de remplissage pour simuler le volume de texte du rap-
port ﬁnal, permettant d’apprécier la mise en page sans être distrait par du latin. Les
avancées technologiques récentes, notamment les architectures Transformers, ont o